# `YourCabs.com`

# Overall Analysis and Implementation Plan
- `ID` - Set as Row Index - All values unique
- `User_ID` - Drop - PII - mobile number - we do not use Personal Information Identifiers in machine learning features
- `vehicle_model_id` - Drop, since >70% of values are 12

- Divide the data into 3 datasets based on travel_type_id, since for different travel types different features are applicable

- After dividing the data into 3 subsets, 3 different machine learning models are to be created:
1. ML Model 1 for travel_type_1
2. ML Model 2 for travel type_2
3. ML Model 3 for travel_type_3

# Below is the column-wise analysis for each subset regarding if they need to be dropped, kept or transformed.

## For travel_type_id = 1

- package_id - drop - all null values
- travel_type_id - drop - all same values
- from_area_id - transform and drop into cancellation volume i.e. Zero, High, Medium or Low cancellation area
- to_area_id - drop - all values null
- from_city_id - use mode to impute the null values
- to_city_id - use mode to impute the null values
- Make a column for inter-city routes - from_city_id -> to_city_id
- city_routes - transform into cancellation volume i.e. Zero, High, Medium or Low cancellation routes
- to_city_id, from_city_id, routes, cancellation_perc - drop
- from_date - transform into - dayOfWeek, Month, Weekday/weekend, TimeofDay(Hour | Morning Afternoon Evening Night)
- time_diff - calculate the number of hours b/w booking and actual start time in hours
- booking_nature - transform the time_diff into Urgent, SameDay, Regular, Advance bookings, since time_diff has >10% outliers
- online_booking | mobile_site_booking - keep
- from_date - drop
- booking_created - drop
- drop all 4 lat/long(from_lat,from_long,to_lat,to_long) as to_lat and to_long has all NULL values and from_lat,from_long's information is already captured in from_area_id


## For travel_type_id = 2

- package_id - drop - all null values
- travel_type_id - drop - all same values
- from_area_id,to_area_id - Make a column for intra-city routes - from_area_id -> to_area_id - transform into cancellation volume i.e. Zero, High, Medium or Low cancellation routes
- from_city_id - drop - all values null
- to_city_id - drop - all values null
- from_date - transform into - dayOfWeek, Month, Weekday/weekend, TimeofDay(Hour | Morning Afternoon Evening Night)
- time_diff - calculate the number of hours b/w booking and actual start time in hours
- booking_nature - transform the time_diff into Urgent, SameDay, Regular, Advance bookings, since time_diff has >10% outliers
- online_booking | mobile_site_booking - keep
- from_date - drop
- booking_created - drop
- from_lat,from_long,to_lat,to_long - impute missing using median
- from these lat/long - calculate the distance in KM (geopy) and then drop all four lat/long col


## For travel_type_id = 3

- package_id - keep
- travel_type_id - drop - all same values
- from_area_id - transform and drop into cancellation volume i.e. Zero, High, Medium or Low cancellation area
- to_area_id - drop - all values null
- from_city_id - drop - all values null
- to_city_id - drop - all values null
- from_date - transform into - dayOfWeek, Month, Weekday/weekend, TimeofDay(Hour | Morning Afternoon Evening Night)
- time_diff - calculate the number of hours b/w booking and actual start time in hours
- booking_nature - transform the time_diff into Urgent, SameDay, Regular, Advance bookings, since time_diff has >10% outliers
- online_booking | mobile_site_booking - keep
- from_date - drop
- booking_created - drop
- drop all 4 lat/long(from_lat,from_long,to_lat,to_long) as to_lat and to_long has all NULL values and from_lat,from_long's information is already captured in from_area_id


## Preprocess these subsets if required such as label encoding etc.


## Machine Learning - Classification
- Decision Tree
- Random Forest
- Naive Bayes Classifier

## Compare the performance and then select the BEST model for each of the three datasets


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('YourCabs.csv')

In [3]:
df.sample(5)

,id,user_id,vehicle_model_id,travel_type_id,package_id,from_area_id,to_area_id,from_city_id,to_city_id,from_date,online_booking,mobile_site_booking,booking_created,from_lat,from_long,to_lat,to_long,Car_Cancellation
2323,135320,23673,12,2,NaN,83.0,393.0,NaN,NaN,1/26/2013 3:00,1,0,1/25/2013 21:12,12.924150,77.672290,13.19956,77.70688,0
44,132566,403,12,2,NaN,1192.0,83.0,NaN,NaN,01-02-2013 07:15,0,0,01-01-2013 19:04,12.976770,77.572700,12.92415,77.67229,0
41865,184007,47733,12,2,NaN,393.0,83.0,15.0,NaN,11-11-2013 10:30,1,0,11-10-2013 22:41,13.199560,77.706880,12.92415,77.67229,0
13435,148967,30772,12,2,NaN,1294.0,1338.0,NaN,NaN,5/13/2013 9:15,1,0,05-12-2013 23:45,12.964653,77.599573,13.03017,77.56684,0
32078,171957,41821,12,2,NaN,410.0,393.0,15.0,NaN,09-07-2013 05:45,0,1,09-06-2013 16:17,13.051210,77.541130,13.19956,77.70688,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43431 entries, 0 to 43430
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   43431 non-null  int64  
 1   user_id              43431 non-null  int64  
 2   vehicle_model_id     43431 non-null  int64  
 3   travel_type_id       43431 non-null  int64  
 4   package_id           7550 non-null   float64
 5   from_area_id         43343 non-null  float64
 6   to_area_id           34293 non-null  float64
 7   from_city_id         16345 non-null  float64
 8   to_city_id           1588 non-null   float64
 9   from_date            43431 non-null  object 
 10  online_booking       43431 non-null  int64  
 11  mobile_site_booking  43431 non-null  int64  
 12  booking_created      43431 non-null  object 
 13  from_lat             43338 non-null  float64
 14  from_long            43338 non-null  float64
 15  to_lat               34293 non-null 

## Set id as Index

In [5]:
df.set_index('id', inplace=True)

In [6]:
df.sample(5)

,user_id,vehicle_model_id,travel_type_id,package_id,from_area_id,to_area_id,from_city_id,to_city_id,from_date,online_booking,mobile_site_booking,booking_created,from_lat,from_long,to_lat,to_long,Car_Cancellation
id,,,,,,,,,,,,,,,,,
155793,34334,12,3,2.0,1239.0,NaN,NaN,NaN,6/20/2013 8:00,1,0,6/19/2013 16:41,12.972510,77.619540,NaN,NaN,0
170069,41044,12,2,NaN,571.0,1328.0,15.0,NaN,8/30/2013 20:00,0,0,8/30/2013 13:52,12.951850,77.696420,12.958808,77.649047,0
172090,41893,12,2,NaN,1341.0,807.0,15.0,NaN,09-07-2013 08:45,1,0,09-07-2013 07:10,12.979703,77.693820,13.012850,77.582740,0
145835,29089,85,2,NaN,448.0,393.0,NaN,NaN,4/24/2013 4:00,0,0,4/23/2013 17:41,12.927320,77.635750,13.199560,77.706880,0
158165,35489,12,2,NaN,1369.0,1096.0,NaN,NaN,07-06-2013 22:15,1,0,07-03-2013 14:38,12.995860,77.596411,12.965190,77.719320,0


## Dropping Duplicates

In [7]:
df.duplicated().sum()

np.int64(41)

In [8]:
# df.drop_duplicates(inplace=True)

In [9]:
print(df.duplicated().sum())

41


In [10]:
# pd.set_option('display.max_rows()',None)
df['vehicle_model_id'].value_counts()

vehicle_model_id
12    31859
85     2407
89     2391
65     1912
28     1702
24     1494
87      565
90      312
23      297
86      123
10      104
64       85
54       73
17       40
91       25
30       14
36        9
13        7
72        2
43        2
1         2
76        1
69        1
14        1
75        1
70        1
39        1
Name: count, dtype: int64

## Dropping user_id and vehicle_model_id

In [11]:
df.drop(columns=['user_id', 'vehicle_model_id'], axis=1, inplace=True)

In [12]:
df.sample(2)

,travel_type_id,package_id,from_area_id,to_area_id,from_city_id,to_city_id,from_date,online_booking,mobile_site_booking,booking_created,from_lat,from_long,to_lat,to_long,Car_Cancellation
id,,,,,,,,,,,,,,,
152358,2,NaN,393.0,1262.0,NaN,NaN,06-06-2013 16:45,1,0,06-01-2013 11:51,13.19956,77.70688,12.815750,77.684650,0
150429,2,NaN,339.0,1351.0,NaN,NaN,5/21/2013 20:00,1,0,5/21/2013 18:33,12.91028,77.64512,12.894352,77.675265,1


## Checking datatypes

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 43431 entries, 132512 to 185941
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   travel_type_id       43431 non-null  int64  
 1   package_id           7550 non-null   float64
 2   from_area_id         43343 non-null  float64
 3   to_area_id           34293 non-null  float64
 4   from_city_id         16345 non-null  float64
 5   to_city_id           1588 non-null   float64
 6   from_date            43431 non-null  object 
 7   online_booking       43431 non-null  int64  
 8   mobile_site_booking  43431 non-null  int64  
 9   booking_created      43431 non-null  object 
 10  from_lat             43338 non-null  float64
 11  from_long            43338 non-null  float64
 12  to_lat               34293 non-null  float64
 13  to_long              34293 non-null  float64
 14  Car_Cancellation     43431 non-null  int64  
dtypes: float64(9), int64(4), object(2)


In [14]:
# df['package_id'] = df['package_id'].astype('int')

# df['from_area_id'] = df['from_area_id'].astype('object')
# df['to_area_id'] = df['to_area_id'].astype('object')
# df['from_city_id'] = df['from_city_id'].astype('object')
# df['to_city_id'] = df['to_city_id'].astype('object')

# df['from_date'] = pd.to_datetime(df['from_date'],formate='mixed')
# df['booking_created'] = pd.to_datetime(df['booking_created'],formate='mixed')

In [15]:
df['travel_type_id'].value_counts()

travel_type_id
2    34292
3     7550
1     1589
Name: count, dtype: int64

## Dividing the datasets into 3 based on travel_type_id

In [16]:
tt1 = df.loc[df.travel_type_id==1]
tt2 = df.loc[df.travel_type_id==2]
tt3 = df.loc[df.travel_type_id==3]

In [17]:
tt1.sample(3)

,travel_type_id,package_id,from_area_id,to_area_id,from_city_id,to_city_id,from_date,online_booking,mobile_site_booking,booking_created,from_lat,from_long,to_lat,to_long,Car_Cancellation
id,,,,,,,,,,,,,,,
147861,1,NaN,1083.0,NaN,NaN,32.0,05-07-2013 06:30,1,0,05-06-2013 23:03,12.900648,77.589183,NaN,NaN,0
136067,1,NaN,1166.0,NaN,NaN,55.0,02-02-2013 00:00,1,0,1/31/2013 20:07,12.997198,77.669514,NaN,NaN,0
167046,1,NaN,1056.0,NaN,15.0,55.0,8/15/2013 7:15,0,0,8/15/2013 6:21,12.919199,77.609310,NaN,NaN,0


In [18]:
tt2.sample(3)

,travel_type_id,package_id,from_area_id,to_area_id,from_city_id,to_city_id,from_date,online_booking,mobile_site_booking,booking_created,from_lat,from_long,to_lat,to_long,Car_Cancellation
id,,,,,,,,,,,,,,,
140947,2,NaN,488.0,585.0,NaN,NaN,3/14/2013 22:15,0,0,03-12-2013 21:03,12.959979,77.647998,12.976770,77.572700,0
140956,2,NaN,730.0,1230.0,NaN,NaN,3/13/2013 10:45,0,0,03-12-2013 22:55,13.033860,77.512690,13.022390,77.594920,0
138676,2,NaN,1153.0,1257.0,NaN,NaN,2/21/2013 8:45,0,0,2/20/2013 21:57,12.871530,77.602060,12.934477,77.611284,0


In [19]:
tt3.sample(3)

,travel_type_id,package_id,from_area_id,to_area_id,from_city_id,to_city_id,from_date,online_booking,mobile_site_booking,booking_created,from_lat,from_long,to_lat,to_long,Car_Cancellation
id,,,,,,,,,,,,,,,
181433,3,6.0,150.0,NaN,NaN,NaN,10/28/2013 7:30,0,0,10/27/2013 22:48,12.897240,77.610650,NaN,NaN,1
172947,3,6.0,1084.0,NaN,NaN,NaN,09-12-2013 07:30,0,0,09-11-2013 22:43,12.892657,77.581866,NaN,NaN,0
168934,3,4.0,271.0,NaN,NaN,NaN,8/24/2013 6:15,0,0,8/24/2013 0:48,12.956410,77.640760,NaN,NaN,0


## Preprocessing `tt1`

In [20]:
tt1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1589 entries, 132519 to 185916
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   travel_type_id       1589 non-null   int64  
 1   package_id           0 non-null      float64
 2   from_area_id         1501 non-null   float64
 3   to_area_id           1 non-null      float64
 4   from_city_id         1393 non-null   float64
 5   to_city_id           1588 non-null   float64
 6   from_date            1589 non-null   object 
 7   online_booking       1589 non-null   int64  
 8   mobile_site_booking  1589 non-null   int64  
 9   booking_created      1589 non-null   object 
 10  from_lat             1496 non-null   float64
 11  from_long            1496 non-null   float64
 12  to_lat               1 non-null      float64
 13  to_long              1 non-null      float64
 14  Car_Cancellation     1589 non-null   int64  
dtypes: float64(9), int64(4), object(2)
m

In [21]:
tt1.drop(columns=[ 'package_id', 'travel_type_id', 'to_area_id','from_lat','from_long', 'to_lat', 'to_long'], axis=1, inplace=True)

In [22]:
tt1.sample(3)

,from_area_id,from_city_id,to_city_id,from_date,online_booking,mobile_site_booking,booking_created,Car_Cancellation
id,,,,,,,,
153619,1189.0,NaN,55.0,06-08-2013 02:00,1,0,06-07-2013 00:02,0
151585,1096.0,15.0,29.0,5/31/2013 6:00,0,0,5/28/2013 16:13,0
151358,571.0,15.0,32.0,06-01-2013 06:00,0,0,5/27/2013 8:41,0


In [23]:
tt1['from_area_id'] = tt1['from_area_id'].astype('object')
tt1['from_city_id'] = tt1['from_city_id'].astype('object')
tt1['to_city_id'] = tt1['to_city_id'].astype('object')

In [24]:
tt1['from_date'] = pd.to_datetime(tt1['from_date'],format='mixed')
tt1['booking_created'] = pd.to_datetime(tt1['booking_created'],format='mixed')

In [25]:
tt1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1589 entries, 132519 to 185916
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   from_area_id         1501 non-null   object        
 1   from_city_id         1393 non-null   object        
 2   to_city_id           1588 non-null   object        
 3   from_date            1589 non-null   datetime64[ns]
 4   online_booking       1589 non-null   int64         
 5   mobile_site_booking  1589 non-null   int64         
 6   booking_created      1589 non-null   datetime64[ns]
 7   Car_Cancellation     1589 non-null   int64         
dtypes: datetime64[ns](2), int64(3), object(3)
memory usage: 111.7+ KB


### Missing Value Treatment

In [26]:
print((tt1.isnull().sum() / len(tt1) * 100).round(2))

from_area_id            5.54
from_city_id           12.33
to_city_id              0.06
from_date               0.00
online_booking          0.00
mobile_site_booking     0.00
booking_created         0.00
Car_Cancellation        0.00
dtype: float64


In [27]:
tt1.describe(include=object)

,from_area_id,from_city_id,to_city_id
count,1501.0,1393.0,1588.0
unique,297.0,3.0,116.0
top,393.0,15.0,32.0
freq,257.0,1281.0,475.0


In [28]:
tt1.from_area_id.fillna(tt1['from_area_id'].mode()[0], inplace=True)
tt1.from_city_id.fillna(tt1.from_city_id.mode()[0], inplace=True)
tt1.to_city_id.fillna(tt1.to_city_id.mode()[0], inplace=True)
# tt1.from_area_id.fillna('393.0',inplace=True)
# tt1.from_city_id.fillna('15.0',inplace=True)
# tt1.to_city_id.fillna('32.0',inplace=True)

In [29]:
tt1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1589 entries, 132519 to 185916
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   from_area_id         1589 non-null   float64       
 1   from_city_id         1589 non-null   float64       
 2   to_city_id           1589 non-null   float64       
 3   from_date            1589 non-null   datetime64[ns]
 4   online_booking       1589 non-null   int64         
 5   mobile_site_booking  1589 non-null   int64         
 6   booking_created      1589 non-null   datetime64[ns]
 7   Car_Cancellation     1589 non-null   int64         
dtypes: datetime64[ns](2), float64(3), int64(3)
memory usage: 111.7 KB


### Checking duplicates

In [30]:
print(tt1.duplicated().sum())

0


### Transform & drop `from_area_id`

In [31]:
from_area_id_cancel_perc = tt1[['from_area_id', 'Car_Cancellation']].groupby('from_area_id').mean()['Car_Cancellation']
tt1['from_area_cancel_perc'] = tt1.from_area_id.map(from_area_id_cancel_perc)
tt1.head(5)

,from_area_id,from_city_id,to_city_id,from_date,online_booking,mobile_site_booking,booking_created,Car_Cancellation,from_area_cancel_perc
id,,,,,,,,,
132519,571.0,15.0,108.0,2013-01-01 09:45:00,0,0,2013-01-01 09:21:00,0,0.020408
132531,1383.0,15.0,32.0,2013-01-12 08:00:00,1,0,2013-01-01 12:17:00,0,0.000000
132539,515.0,15.0,32.0,2013-01-05 08:30:00,0,0,2013-01-01 14:40:00,0,0.000000
132697,136.0,1.0,152.0,2013-01-04 09:30:00,1,0,2013-01-03 12:19:00,0,0.000000
132760,1118.0,15.0,32.0,2013-01-06 07:30:00,1,0,2013-01-04 09:32:00,0,0.000000


In [32]:
tt1.loc[tt1.from_area_cancel_perc>0, 'from_area_cancel_perc']

id
132519    0.020408
132907    0.008696
132911    0.076923
133098    0.008696
133312    0.166667
            ...   
185701    0.020408
185732    0.008696
185738    0.020408
185843    0.008696
185899    0.008696
Name: from_area_cancel_perc, Length: 610, dtype: float64

In [33]:
tt1.loc[tt1.from_area_cancel_perc>0, 'from_area_cancel_perc'].describe()

count    610.000000
mean       0.034426
std        0.066119
min        0.008696
25%        0.008696
50%        0.008696
75%        0.021277
max        0.500000
Name: from_area_cancel_perc, dtype: float64

In [34]:
tt1.loc[tt1.from_area_cancel_perc>0, 'from_area_cancel_perc'].quantile(0.33)

np.float64(0.008695652173913044)

In [35]:
print(tt1.loc[tt1.from_area_cancel_perc>0, 'from_area_cancel_perc'].quantile(0.66))

0.02040816326530612


In [36]:
tt1['from_area_cancel_vol'] = np.where(tt1.from_area_cancel_perc > 0.0204081, "High", 
                                      np.where(tt1.from_area_cancel_perc == 0, "Zero",
                                      np.where(tt1.from_area_cancel_perc < 0.008695, "Low","Medium")))

In [37]:
tt1.sample(5)

,from_area_id,from_city_id,to_city_id,from_date,online_booking,mobile_site_booking,booking_created,Car_Cancellation,from_area_cancel_perc,from_area_cancel_vol
id,,,,,,,,,,
166620,142.0,15.0,32.0,2013-08-14 07:45:00,1,0,2013-08-13 13:49:00,0,0.000000,Zero
137863,793.0,15.0,25.0,2013-02-15 08:30:00,0,0,2013-02-14 21:14:00,0,0.000000,Zero
183977,693.0,15.0,32.0,2013-11-11 06:00:00,1,0,2013-11-10 19:50:00,0,0.000000,Zero
174843,1155.0,15.0,12.0,2013-09-22 07:30:00,0,0,2013-09-21 19:15:00,0,0.000000,Zero
145211,571.0,15.0,89.0,2013-04-19 02:00:00,0,0,2013-04-19 01:15:00,0,0.020408,High


In [38]:
tt1.from_area_cancel_vol.value_counts()

from_area_cancel_vol
Zero      979
Medium    345
High      265
Name: count, dtype: int64

In [39]:
# Now we can drop from_area_id and from_area_cancel_perc
tt1.drop(['from_area_id', 'from_area_cancel_perc'], axis=1, inplace=True)

In [40]:
tt1.sample(4)

,from_city_id,to_city_id,from_date,online_booking,mobile_site_booking,booking_created,Car_Cancellation,from_area_cancel_vol
id,,,,,,,,
147611,15.0,161.0,2013-05-06 05:30:00,0,0,2013-05-05 09:26:00,0,Zero
182881,15.0,32.0,2013-11-03 04:30:00,0,0,2013-11-02 22:22:00,0,Zero
146479,15.0,55.0,2013-04-28 09:30:00,0,0,2013-04-28 04:17:00,0,Medium
179222,15.0,152.0,2013-10-16 07:00:00,0,0,2013-10-15 11:08:00,0,Zero


### Transfrom & drop `from_city_id` and `to_city_id`

In [41]:
tt1['city_route'] = tt1.from_city_id.astype('str')+ '->'+ tt1.to_city_id.astype('str')

In [42]:
tt1.sample(3)

,from_city_id,to_city_id,from_date,online_booking,mobile_site_booking,booking_created,Car_Cancellation,from_area_cancel_vol,city_route
id,,,,,,,,,
145505,15.0,24.0,2013-04-21 08:30:00,0,0,2013-04-21 07:49:00,0,Zero,15.0->24.0
133653,15.0,24.0,2013-01-13 08:00:00,0,0,2013-01-12 15:41:00,0,Zero,15.0->24.0
166427,15.0,32.0,2013-08-13 07:30:00,0,0,2013-08-12 15:26:00,0,Zero,15.0->32.0


In [43]:
city_route_cancel_perc = tt1[['city_route', 'Car_Cancellation']].groupby('city_route').mean()['Car_Cancellation']
tt1['city_route_cancel_perc'] = tt1.city_route.map(city_route_cancel_perc)
tt1.sample(3)

,from_city_id,to_city_id,from_date,online_booking,mobile_site_booking,booking_created,Car_Cancellation,from_area_cancel_vol,city_route,city_route_cancel_perc
id,,,,,,,,,,
159446,1.0,32.0,2013-07-14 06:00:00,1,0,2013-07-09 00:06:00,0,Medium,1.0->32.0,0.0
179222,15.0,152.0,2013-10-16 07:00:00,0,0,2013-10-15 11:08:00,0,Zero,15.0->152.0,0.0
145731,15.0,44.0,2013-04-23 13:00:00,0,0,2013-04-22 16:59:00,0,Zero,15.0->44.0,0.0


In [44]:
print(tt1.loc[tt1.city_route_cancel_perc>0, 'city_route_cancel_perc'].quantile(0.33))

0.017937219730941704


In [45]:
print(tt1.loc[tt1.city_route_cancel_perc>0, 'city_route_cancel_perc'].quantile(0.66))

0.017937219730941704


In [46]:
tt1.loc[tt1.city_route_cancel_perc>0, 'city_route_cancel_perc'].describe()

count    793.000000
mean       0.026482
std        0.019527
min        0.012048
25%        0.017937
50%        0.017937
75%        0.037037
max        0.086957
Name: city_route_cancel_perc, dtype: float64

In [47]:
tt1['city_route_cancel_vol'] = np.where(tt1.city_route_cancel_perc > 0.018, "High", 
                                      np.where(tt1.city_route_cancel_perc == 0, "Zero",
                                      np.where(tt1.city_route_cancel_perc < 0.017, "Low","Medium")))

In [48]:
tt1.sample(3)

,from_city_id,to_city_id,from_date,online_booking,mobile_site_booking,booking_created,Car_Cancellation,from_area_cancel_vol,city_route,city_route_cancel_perc,city_route_cancel_vol
id,,,,,,,,,,,
141456,15.0,190.0,2013-03-18 12:00:00,0,0,2013-03-18 11:34:00,0,Zero,15.0->190.0,0.000000,Zero
155088,15.0,32.0,2013-06-18 06:00:00,1,0,2013-06-15 01:48:00,0,Zero,15.0->32.0,0.017937,Medium
161483,15.0,58.0,2013-07-21 12:45:00,0,0,2013-07-19 16:02:00,0,Medium,15.0->58.0,0.000000,Zero


In [49]:
tt1.drop(['from_city_id', 'to_city_id', 'city_route', 'city_route_cancel_perc'],axis=1, inplace=True)

In [51]:
tt1.sample(5)

,from_date,online_booking,mobile_site_booking,booking_created,Car_Cancellation,from_area_cancel_vol,city_route_cancel_vol
id,,,,,,,
183670,2013-12-06 11:00:00,1,0,2013-11-08 18:19:00,0,Medium,Zero
158242,2013-07-04 09:00:00,0,0,2013-07-03 20:50:00,0,High,Zero
154360,2013-06-15 07:45:00,0,0,2013-06-10 18:56:00,0,Medium,Zero
162053,2013-07-23 08:00:00,0,0,2013-07-22 12:34:00,0,Zero,Low
147419,2013-05-04 14:00:00,0,0,2013-05-03 20:43:00,0,Medium,Medium
